In [20]:
"""
    generate a 18kHz to 22kHz ofdm wav signal. each chirp takes 2.9583 milliseconds, repeated 1000 times.
"""

import math
import wave
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import sys
import time
# from commpy.filters import rrcosfilter

#define the params of wave
channels = 1
file_name = './ofdm_44100_17822khz_257_paprconfirm'
#define the time of one wave
# time = 0.02
intv = 0


K = 257 # number of OFDM subcarriers
Kover = K
nframes = round(400000*128/K)
CP = round(Kover/3.5) #print(CP);  # length of the cyclic prefix: 25% of the block
mu = 3 # bits per symbol (i.e. QPSK)
pilotAmplitude=1
ntest = 2000000

framerate = 44100
fd2000 = framerate/2000
subc_begin = int(np.ceil((17.8/fd2000*Kover)))
subc_end = round(22/fd2000*Kover)
nData = subc_end-subc_begin+1 #round(22/128*K)#round(3/16*K)
mQAM=mu
pilotCarriers = np.arange(Kover)  # indices of all subcarriers ([0, 1, ... K-1])
payloadBits_per_OFDM = nData*mu  # number of payload bits per OFDM symbol

sampwidth = 4
ampli = 34000000*Kover #200000000*Kover

oversam = Kover//K

if mu ==1:
  mapping_table = {
  (0,) : -1,
  (1,) : +1,
  (2,) : 0
  }
elif mu==2:
    mapping_table = {
    (0,0) : -1-1j,
    (0,1) : -1+1j,
    (1,0) :  1-1j,
    (1,1) :  1+1j,
    (2,2) : 0,(1,2) : 0,(2,1) : 0,(0,2) : 0,(2,0) : 0
    }
elif mu==3:
  mapping_table = {
     (0,0,0) : np.sqrt(2),(0,0,1) : 1+1j, 
    (0,1,0) : 1.41421356j,(0,1,1) : -1+1j,
    (1,0,0) : -1.41421356, (1,0,1) : -1-1j,
    (1,1,0) : -1.41421356j, (1,1,1) : 1-1j,
    (2,2,2) : 0,(1,2,2) : 0,(2,1,2) : 0,(0,2,2) : 0,(2,0,2) : 0}

bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM, ))
print ("Bits count: ", len(bits))
#print ("First 20 bits: ", bits[:64])
print ("Mean of bits (should be around 0.5): ", np.mean(bits))


def SP(bits):
  lownullbit=np.full((subc_begin*mu, ),2,dtype=int)
  highnullbit = np.full(((Kover-subc_begin-nData*oversam)*mu, ),2,dtype=int)
  bits=np.concatenate((lownullbit,bits,highnullbit),axis=None)
  return bits.reshape((len(pilotCarriers), mu))
def Mapping(bits):
  return np.array([mapping_table[tuple(b)] for b in bits])
def OFDM_symbol(QAM_payload):
  symbol = np.zeros(Kover, dtype=complex) # the overall K subcarriers
  symbol[pilotCarriers] = QAM_payload  # allocate the pilot subcarriers 
  #symbol[dataCarriers] = QAM_payload  # allocate the pilot subcarriers
  return symbol
def IFFT(OFDM_data):
  OFDM_data2=np.append(OFDM_data,np.conj(OFDM_data[-2:0:-1])) #[0] is the DC component and [-1] is the symmetric
  fft=np.fft.irfft(OFDM_data) # for QPSK
#   hfft=np.fft.ihfft(OFDM_data2) # for BPSK
  return fft
def addCP(OFDM_time,index):
  pre_win = np.arange(CP)
  prewin = [math.cos(math.radians(pre*90/np.max(CP)-90))**2 for pre in pre_win]
  sufwin = prewin[::-1]
  if index>-1:
    cp = OFDM_time[-CP:]*prewin               # take the last CP samples ...
    cf = OFDM_time[:CP]*sufwin
    return np.hstack([cp, OFDM_time,cf])  # ... and add them to the beginning
  else: 
    return np.hstack([OFDM_time[-CP:], OFDM_time])
def addWindow(OFDM_time, alpha=0.5):
  tukey_win = signal.tukey(len(OFDM_time),alpha)
  return OFDM_time*tukey_win

def OFDM(i):
  if isinstance(i, np.ndarray):
    bits = i
    i = -3
  elif intv==0 and i==-2:
    bits=np.array([1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,1,0,1,1,1,1,0,1
,0,0,1,0,1,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1
,0,1,1,1,1,0,1,0,1,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,0,1,0,1,1,0,0,1,0,1
,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,0,1
,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0
,1,1,0,0,1,1,0,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,0,1,1,1,0,0,1,0,1,0,0,1,0
,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1
,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,1
,0,1,0,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1
,1,0,1,0,0,0,0,0,1,1,0,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,1,0,0,1,1
,0,0,1,1,0,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1
,1,0,1,1,0,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,0,0
,1,1,1,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0
,1,0,0,0,0,1,1,1,1,0,1,1,0,1,1])
    # bits=np.array([0,1,0,0,0,1,1,0,1,1,1,1,1,0,0,0,1,0,1,0,0,1])
  else:
    bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM, ))
  bitssave=bits
#   Insert every n points, for time prolong
#   osam = np.full((payloadBits_per_OFDM*(oversam), ),2,dtype=int)
#   osam[::mu*(oversam)]=bits[::mu]
# #   osam[1::mu*(oversam)]=bits[1::mu]
#   bits = osam
  bits_SP = SP(bits)
  QAM = Mapping(bits_SP)
  demapping_table = {v : k for k, v in mapping_table.items()}
  OFDM_data = OFDM_symbol(QAM)
  # print ("Number of OFDM carriers in frequency domain: ", len(OFDM_data))
  OFDM_time = IFFT(OFDM_data)*ampli

  # print ("Number of OFDM samples in time-domain before CP: ", len(OFDM_time))
  if intv==0 and i<=0:
    OFDM_withCP = OFDM_time
  else:
    # OFDM_withCP = addCP(OFDM_time,i)
    OFDM_withCP = addWindow(OFDM_time)
  return np.real(OFDM_withCP), bitssave#OFDM_data[subc_begin:subc_begin+nData].reshape(nData,)
  # print ("Number of OFDM samples in time domain with CP: ", len(OFDM_withCP))

def peak_to_sidelobe_ratio(ofdm_frame):
    correlation = np.correlate(np.concatenate((ofdm_frame, ofdm_frame)), ofdm_frame, mode='valid')
    correlation = correlation[:len(ofdm_frame)]
    peak = np.max(correlation)
    sidelobes = np.delete(correlation, np.argmax(correlation))
    sidelobe_max = np.max(sidelobes)
    psr = peak / sidelobe_max
    plt.plot(np.arange(len(correlation)), correlation)
    plt.show()
    return psr

dataone,bitsone=OFDM(0)
# print ("First 20 bits: ", bits[:64])
interval = np.zeros(int(intv*oversam),)
rt_data = np.concatenate((interval,dataone,np.zeros(200,)))
rt_bits = bitsone
ofdm_data, ofdm_bits = OFDM(0)

print("The starting index, ending index, and ending index are: ",subc_begin,", ",subc_end,", ",nData)

####### calculate papr
papr_one = np.array(max(abs(dataone))/np.sqrt(np.mean(dataone**2)))
rt_bits = np.ones([ntest+1, len(bitsone)])*-2
rt_bits[0] = bitsone
rt_papr = np.ones(ntest+1)*10
rt_papr[0] = papr_one
rt_pspr = np.zeros(ntest+1)*10
# rt_pspr[0] = peak_to_sidelobe_ratio(dataone)


Bits count:  147
Mean of bits (should be around 0.5):  0.4557823129251701
The starting index, ending index, and ending index are:  208 ,  256 ,  49


In [ ]:
t_start = time.time()
for i in range(ntest):
    datatwo,bitstwo=OFDM(0)    
    paprtwo = max(abs(datatwo))/np.sqrt(np.mean(datatwo**2))
    rt_papr[i+1] = paprtwo
    rt_bits[i+1] = bitstwo
    if i%10000==0:
      print(f"Progress: {i}/{ntest} at time of {time.time() - t_start}")
rt_index = rt_papr
lowestbits = rt_bits[np.where(rt_index == np.min(rt_index))[0][0],:]
print(lowestbits)
print("The lowest PAPR is: ",np.min(rt_papr))
print("The greatest PASR is: ",np.max(rt_pspr))
# print the rt_papr to a csv file
# np.savetxt('./'+ file_name+'.csv', rt_papr, delimiter=',')
np.savetxt('./'+ file_name+'_bits.csv', rt_bits, delimiter=',')


In [23]:
lowestbits = np.loadtxt('./'+ file_name+'_bits.csv', delimiter=',')
ofdm_data, ofdm_bits = OFDM(lowestbits)
######## Writes Audio Frame
frame_data=np.concatenate((ofdm_data,interval))
frames_data=np.tile(frame_data,nframes)
output_data = np.concatenate((rt_data,frames_data))

if channels==2:
    output_data_zero=np.zeros(2*len(output_data))
    # 1 for right, 0 for left
    output_data_zero[::2]=output_data
    output_data = output_data_zero.astype(np.int32)
    # rt_data_zero = '\0'*nframes*sampwidth
    # rt_data = rt_data_raw.tostring()+rt_data_zero
else:
    output_data = output_data.astype(np.int32)

f = wave.open(file_name+'.wav',"wb")
f.setnchannels(channels)
f.setsampwidth(sampwidth)
f.setframerate(framerate)
f.writeframes(output_data.tobytes())
f.close()

length_frame = len(frame_data) / framerate * 1000
print("The length of one frame is: ", length_frame, "ms")
nframes = 2
one_frame_data = np.concatenate((rt_data,np.tile(frame_data,nframes)))
one_frame_data = one_frame_data.astype(np.int32)
f = wave.open(file_name+f'_{nframes}.wav',"wb")
f.setnchannels(channels)
f.setsampwidth(sampwidth)
f.setframerate(framerate)
f.writeframes(one_frame_data.tobytes())
f.close()


The length of one frame is:  11.609977324263038 ms
